In [4]:
pip install qiskit_optimization

  Using cached qiskit_optimization-0.3.2-py3-none-any.whl (154 kB)
  Using cached qiskit_terra-0.19.2-cp37-cp37m-macosx_10_9_x86_64.whl (5.6 MB)
  Using cached docplex-2.23.222-py3-none-any.whl
  Using cached symengine-0.9.2-cp37-cp37m-macosx_10_9_x86_64.whl (17.7 MB)
  Using cached tweedledum-1.1.1-cp37-cp37m-macosx_10_15_x86_64.whl (951 kB)
  Using cached stevedore-3.5.0-py3-none-any.whl (49 kB)
  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Using cached pbr-5.8.1-py2.py3-none-any.whl (113 kB)
  Using cached zipp-3.7.0-py3-none-any.whl (5.3 kB)
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.17.4
    Uninstalling qiskit-terra-0.17.4:
      Successfully uninstalled qiskit-terra-0.17.4
  Attempting uninstall: docplex
    Found existing installation: docplex 2.15.194
    Uninstalling docplex-2.15.194:
      Successfully uninstalled docplex-2.15.194
ERROR: pip's dependency resolver does not currently take into account all the pack

In [39]:
# Import auxiliary libraries
import numpy as np
# Import Qiskit
# from qiskit import IBMQ, Aer
# from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
# from qiskit.utils import QuantumInstance, algorithm_globals
# from qiskit.providers.aer.noise.noise_model import NoiseModel
from qiskit_optimization import QuadraticProgram
# from qiskit_optimization.algorithms import MinimumEigenOptimizer
# from qiskit_optimization.converters import QuadraticProgramToQubo
# import qiskit.test.mock as Fake

In [40]:
n=3 #number of power plants
N=5 #number of discrete states
A =[10,20,30]
B =[40,50,60]
C =[7,8,9]
p_min=[1,2,3]
p_max=[4,5,6]
h=[]
qindxs = {}

for i in range(len(p_min)):
    h.append((p_max[i]-p_min[i])/N)

A_scale = 100 #A is the constant for the constraint for the cost function
B_scale = 50 #B is the constant for the costraint for the cost function
L = 100

In [41]:
def zindx(i,k):
    return 'xz%s%s' %(i,k)

def vindx(i):
    return 'xv%s' %i

In [42]:
qubo = QuadraticProgram(name="energy")
print(qubo.binary_var_dict(n,key_format="v{}"))
for i in range(n):
    print(qubo.binary_var_dict(key_format='z'+str(i)+'{}',keys=list(range(N+1))))

linear_terms = {}
quadratic_terms = {}

{'xv0': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f892b0>, 'xv1': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89048>, 'xv2': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89470>}
{'xz00': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89550>, 'xz01': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89630>, 'xz02': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f896a0>, 'xz03': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f897b8>, 'xz04': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89828>, 'xz05': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89898>}
{'xz10': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89908>, 'xz11': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f899e8>, 'xz12': <qiskit_optimization.problems.variable.Variable object at 0x7f9710f89978>, 'xz1

In [43]:
# A_i (1-v_i)
for i in range(n):
    linear_terms[vindx(i)] = -A[i]

# B_i p_i
for i in range(n):    
    for k in range(0,N+1):
        linear_terms[zindx(i,k)] = B[i]*(p_min[i]+ k*h[i])
    
        
# C_i p_i^2
for i in range(n):    
    for k in range(0,N+1):
        for m in range(k,N+1):
            if m == k:
                linear_terms[zindx(i,k)] = linear_terms.get(zindx(i,k),0) + linear_terms.get(zindx(i,k),0) + C[i]*(p_min[i] + k*h[i])**2
                continue
            label = (zindx(i,k), zindx(i,m))
            quadratic_terms[label] = quadratic_terms.get(label,0) + 2*C[i]*(p_min[i] + k*h[i])*(p_min[i] + m*h[i])

# v_i + sum_k z_ik = 1
for i in range(n):    
    linear_terms[vindx(i)] = linear_terms.get(vindx(i), 0) - A_scale
    for k in range(0,N+1):
        linear_terms[zindx(i,k)] += -2*A_scale
        label = (vindx(i), zindx(i,k))
        quadratic_terms[label] =  quadratic_terms.get(label, 0) + 2*A_scale
        for m in range(k, N+1):
            if m == k:
                linear_terms[zindx(i,k)] = 2*A_scale
                continue
                
            label = (zindx(i,k), zindx(i,m))
            quadratic_terms[label] += 4*A_scale
            
# sum_i p_i = L
for i in range(n):  
    for k in range(0,N+1):        
        linear_terms[zindx(i,k)] += -2*B_scale*L*(p_min[i] + k*h[i])
        for j in range(i,n):
            for m in range(k,N+1):
                if i == j and m == k:
                    linear_terms[zindx(i,k)] = B_scale*(p_min[i] + k*h[i])**2
                    continue
                label = (zindx(i,k), zindx(j,m))
                quadratic_terms[label]  =  quadratic_terms.get(label, 0) + 4*B_scale*(p_min[i] + k*h[i])*(p_min[j] + m*h[j])        

In [45]:
linear_terms, quadratic_terms

({'xv0': -110,
  'xv1': -120,
  'xv2': -130,
  'xz00': 50.0,
  'xz01': 128.00000000000003,
  'xz02': 242.00000000000003,
  'xz03': 391.99999999999994,
  'xz04': 577.9999999999999,
  'xz05': 800.0,
  'xz10': 200.0,
  'xz11': 338.00000000000006,
  'xz12': 512.0000000000001,
  'xz13': 722.0,
  'xz14': 968.0000000000001,
  'xz15': 1250.0,
  'xz20': 450.0,
  'xz21': 648.0,
  'xz22': 882.0,
  'xz23': 1152.0,
  'xz24': 1458.0000000000002,
  'xz25': 1800.0},
 {('xz00', 'xz01'): 742.4,
  ('xz00', 'xz02'): 870.8000000000001,
  ('xz00', 'xz03'): 999.2,
  ('xz00', 'xz04'): 1127.6,
  ('xz00', 'xz05'): 1256.0,
  ('xz01', 'xz02'): 1153.28,
  ('xz01', 'xz03'): 1358.72,
  ('xz01', 'xz04'): 1564.16,
  ('xz01', 'xz05'): 1769.6,
  ('xz02', 'xz03'): 1718.24,
  ('xz02', 'xz04'): 2000.7200000000003,
  ('xz02', 'xz05'): 2283.2000000000003,
  ('xz03', 'xz04'): 2437.2799999999997,
  ('xz03', 'xz05'): 2796.8,
  ('xz04', 'xz05'): 3310.4,
  ('xz10', 'xz11'): 1523.2,
  ('xz10', 'xz12'): 1782.4,
  ('xz10', 'xz13'): 

In [33]:
qubo.minimize(linear=linear_terms,quadratic=quadratic_terms)

In [34]:
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: energy

Minimize
 obj: - 110 xv0 - 120 xv1 - 130 xv2 + 50 xz00 + 128.000000000000 xz01
      + 242.000000000000 xz02 + 392.000000000000 xz03 + 578.000000000000 xz04
      + 800 xz05 + 200 xz10 + 338.000000000000 xz11 + 512.000000000000 xz12
      + 722 xz13 + 968.000000000000 xz14 + 1250 xz15 + 450 xz20 + 648 xz21
      + 882 xz22 + 1152 xz23 + 1458.000000000000 xz24 + 1800 xz25 + [
      400 xv0*xz00 + 400 xv0*xz01 + 400 xv0*xz02 + 400 xv0*xz03 + 400 xv0*xz04
      + 400 xv0*xz05 + 400 xv1*xz10 + 400 xv1*xz11 + 400 xv1*xz12 + 400 xv1*xz13
      + 400 xv1*xz14 + 400 xv1*xz15 + 400 xv2*xz20 + 400 xv2*xz21 + 400 xv2*xz22
      + 400 xv2*xz23 + 400 xv2*xz24 + 400 xv2*xz25 + 1484.800000000000 xz00*xz01
      + 1741.600000000000 xz00*xz02 + 1998.400000000000 xz00*xz03
      + 2255.200000000000 xz00*xz04 + 2512 xz00*xz05 + 800 xz00*xz10
      + 1040 xz00*xz11 + 1280 xz00*xz12 + 1520 xz00*xz13
      + 1760.00000000